In [1]:
from sponge import Sponge
from sponge import Molecule
from sponge import ForceField
from sponge import UpdaterMD
from sponge import WithEnergyCell
from sponge.control import VelocityVerlet, Langevin, BerendsenBarostat
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo
from sponge.optimizer import SteepestDescent
from mindspore import context

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])

Parameter (name=pbc_box, shape=(1, 3), dtype=Float32, requires_grad=True)

如果定义了周期性的盒子，那么还可以使用repeat_box功能来扩展/复制系统，比如可以这样将系统复制1000份

In [4]:
system.repeat_box([10, 10, 10])

Molecule<>

得到的分子系统的示意图如下所示：
![](../../docs/tutorial_b05.png)

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = SteepestDescent(system.trainable_params(), 1e-6)

In [7]:
sim = WithEnergyCell(system, potential, cutoff=1.0)
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Calculate the neighbour list using the inter-atomic distances.


[WARNING] KERNEL(1445437,7fbdb7a3a500,python):2023-05-19-09:53:00.104.294 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel.cc:40] CheckDeviceSm] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6


In [8]:
run_info = RunInfo(10)
md.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-19 09:53:07
[MindSPONGE] Step: 0, E_pot: -7163.629
[MindSPONGE] Step: 10, E_pot: -7715.919
[MindSPONGE] Step: 20, E_pot: -7753.0566
[MindSPONGE] Step: 30, E_pot: -7755.648
[MindSPONGE] Step: 40, E_pot: -7755.898
[MindSPONGE] Step: 50, E_pot: -7755.876
[MindSPONGE] Step: 60, E_pot: -7756.2266
[MindSPONGE] Step: 70, E_pot: -7756.2534
[MindSPONGE] Step: 80, E_pot: -7756.245
[MindSPONGE] Step: 90, E_pot: -7756.242
[MindSPONGE] Finished simulation at 2023-05-19 09:53:16
[MindSPONGE] Simulation time: 8.39 seconds.
--------------------------------------------------------------------------------


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, temp),
                barostat=BerendsenBarostat(system, 1),
                velocity=velocity,
                time_step=5e-4)
md = Sponge(sim, optimizer=opt)

In [11]:
md.change_optimizer(opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b05.h5md', save_freq=10)

In [13]:
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-19 09:53:22
[MindSPONGE] Step: 0, E_pot: -7756.253, E_kin: 11301.678, E_tot: 3545.4248, Temperature: 302.16284, Pressure: 649.8786, Volume: 64.0
[MindSPONGE] Step: 10, E_pot: -6364.685, E_kin: 10050.27, E_tot: 3685.5845, Temperature: 268.70505, Pressure: -1139.3953, Volume: 63.990913
[MindSPONGE] Step: 20, E_pot: -7306.667, E_kin: 11098.0205, E_tot: 3791.3535, Temperature: 296.7178, Pressure: 380.5648, Volume: 63.978497
[MindSPONGE] Step: 30, E_pot: -6612.3, E_kin: 10596.975, E_tot: 3984.6748, Temperature: 283.3218, Pressure: -1579.7797, Volume: 63.966797
[MindSPONGE] Step: 40, E_pot: -7663.0093, E_kin: 11613.1, E_tot: 3950.0903, Temperature: 310.48904, Pressure: -333.62012, Volume: 63.947372
[MindSPONGE] Step: 50, E_pot: -8058.2236, E_kin: 12113.102, E_tot: 4054.878, Temperature: 323.85715, Pressure: -2688.4717, Volume: 63.924316
[MindSPONGE] Step: 60, E_pot: -9006.488, E_kin: 12985.721, E_tot: 3979.2324, Temperature: 347.18756, Pressure: -11